# Slack Bot

## Load libraries

In [2]:
import os
from csv import writer
import pandas as pd
from slack import WebClient
from datetime import date
from slack.errors import SlackApiError
pd.set_option('display.max_columns', None)

/Users/aadittambe/.local/share/virtualenvs/slack-bot-GaVTNlXs/lib/python3.10/site-packages/slack/deprecation.py:16: UserWarning: slack package is deprecated. Please use slack_sdk.web/webhook/rtm package instead. For more info, go to https://slack.dev/python-slack-sdk/v3-migration/
  warnings.warn(message)
/Users/aadittambe/.local/share/virtualenvs/slack-bot-GaVTNlXs/lib/python3.10/site-packages/slack/deprecation.py:16: UserWarning: slack.errors package is deprecated. Please use slack_sdk.errors package instead. For more info, go to https://slack.dev/python-slack-sdk/v3-migration/
  warnings.warn(message)


## Slack options

In [2]:
slack_token = os.environ["SLACK_API_TOKEN"]
client = WebClient(token=slack_token)

## Load data 

In [3]:
data = pd.read_json('https://healthdata.gov/resource/6xf2-c3ie.json')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Columns: 116 entries, state to total_patients_hospitalized_confirmed_influenza_coverage
dtypes: float64(29), int64(85), object(2)
memory usage: 49.1+ KB


## Clean data

In [4]:
data = data[data.state != 'AS' ]
data = data[data.state != 'PR' ]
data = data[data.state != 'VI' ]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 0 to 53
Columns: 116 entries, state to total_patients_hospitalized_confirmed_influenza_coverage
dtypes: float64(29), int64(85), object(2)
memory usage: 46.6+ KB


## National values analysis

### Covid beds

In [5]:
total_beds = data['inpatient_beds'].sum()
covid_occupied_beds = data['inpatient_beds_used_covid'].sum()
percent_covid_beds = round(((covid_occupied_beds/total_beds)*100),2)

### Staffing

In [6]:
critical_staffing_shortage_today_yes = data['critical_staffing_shortage_today_yes'].sum()
critical_staffing_shortage_today_no = data['critical_staffing_shortage_today_no'].sum()
critical_staffing_shortage_today_not_reported = data['critical_staffing_shortage_today_not_reported'].sum()
total_hospitals = critical_staffing_shortage_today_yes + critical_staffing_shortage_today_no + critical_staffing_shortage_today_not_reported

percent_hospitals_critical_shortages = round(((critical_staffing_shortage_today_yes/total_hospitals)*100),2)

today = date.today()
### Write out to a national CSV

In [88]:
head = ["date","total_beds","covid_occupied_beds","percent_covid_beds","critical_staffing_shortage_today_yes","critical_staffing_shortage_today_no","critical_staffing_shortage_today_not_reported","critical_staffing_shortage_today_yes","percent_hospitals_critical_shortages"]
today = date.today()
row = [today,total_beds,covid_occupied_beds,percent_covid_beds,critical_staffing_shortage_today_yes,critical_staffing_shortage_today_no,critical_staffing_shortage_today_not_reported,critical_staffing_shortage_today_yes,percent_hospitals_critical_shortages]
with open('us_data.csv', 'a',encoding='utf8', newline='') as f_object:  
    writer_object = writer(f_object)
    writer_object.writerow(row)  
    f_object.close()

## Maryland values analysis

### Covid beds

In [61]:
md = data[data.state == "MD"]

In [62]:
md_beds = int(md['inpatient_beds'])
md_covid_occupied_beds = int(md['inpatient_beds_used_covid'])
md_percent_covid_beds = round(((md_covid_occupied_beds/md_beds)*100),2)

### Md. staffing

In [90]:
md_critical_staffing_shortage_today_yes = int(md['critical_staffing_shortage_today_yes'])
md_critical_staffing_shortage_today_no = int(md['critical_staffing_shortage_today_no'])
md_critical_staffing_shortage_today_not_reported = int(md['critical_staffing_shortage_today_not_reported'])
md_total_hospitals = md_critical_staffing_shortage_today_yes + md_critical_staffing_shortage_today_no + md_critical_staffing_shortage_today_not_reported

md_percent_hospitals_critical_shortages = round(((md_critical_staffing_shortage_today_yes/md_total_hospitals)*100),2)

### Write out Maryland CSV

In [93]:
head = ["date","md_beds","md_covid_occupied_beds","md_percent_covid_beds","md_critical_staffing_shortage_today_yes","md_critical_staffing_shortage_today_no","md_critical_staffing_shortage_today_not_reported","md_critical_staffing_shortage_today_yes","md_percent_hospitals_critical_shortages"]
row = [today,md_beds,md_covid_occupied_beds,md_percent_covid_beds,md_critical_staffing_shortage_today_yes,md_critical_staffing_shortage_today_no,md_critical_staffing_shortage_today_not_reported,md_critical_staffing_shortage_today_yes,md_percent_hospitals_critical_shortages]
with open('md_data.csv', 'a',encoding='utf8', newline='') as f_object:  
    writer_object = writer(f_object)
    writer_object.writerow(row)  
    f_object.close()

## National-Md. Comparison analysis

### Compare beds

In [67]:
if md_percent_covid_beds > percent_covid_beds:
    beds_above_below = "above"
elif md_percent_covid_beds < percent_covid_beds:
    beds_above_below = "below"
elif md_percent_covid_beds == percent_covid_beds:
    beds_above_below = "the same as"

### Compare staffing levels

In [76]:
if md_percent_hospitals_critical_shortages > percent_hospitals_critical_shortages:
    shortage_below_above = "above"
elif md_percent_hospitals_critical_shortages < percent_hospitals_critical_shortages:
    shortage_below_above = "below"
elif md_percent_hospitals_critical_shortages == percent_hospitals_critical_shortages:
    shortage_below_above = "the same as"

## Past seven days comparison

### National

In [115]:
with open('us_data.csv', 'r') as f: 
    today_line = f.readlines()[-1]
    today_line_list = today_line.split(',')
    try: 
        seven_days_ago_line = f.readlines()[-7]
    except:
        seven_days_ago_line = "Not enough data to report"

In [120]:
today_critical_shortage_hospitals = today_line_list[-1].strip()
if seven_days_ago_line == "Not enough data to report":
    week_ago_critical_shortage_hospitals = "Not enough data to report"
else:
    week_ago_critical_shortage_hospitals = seven_days_ago_line[-7].strip()

### Md.

In [123]:
with open('md_data.csv', 'r') as f: 
    today_line = f.readlines()[-1]
    today_line_list = today_line.split(',')
    try: 
        md_seven_days_ago_line = f.readlines()[-7]
    except:
        md_seven_days_ago_line = "Not enough data to report"

In [124]:
md_today_critical_shortage_hospitals = today_line_list[-1].strip()
if md_seven_days_ago_line == "Not enough data to report":
    md_week_ago_critical_shortage_hospitals = "Not enough data to report"
else:
    md_week_ago_critical_shortage_hospitals = md_seven_days_ago_line[-7].strip()

## Write out message

In [140]:
msg = f"""📢 Daily COVID-19 hospitalization update 📢
Out of {(format (total_beds, ',d'))} available hospital beds in the country, {(format(covid_occupied_beds, ',d'))} beds are occupied by COVID-19 patients today, which account for {percent_covid_beds} percent of all beds.
{percent_hospitals_critical_shortages} percent hospitals in the country are reporting critical staffing shortages today.\n
Maryland reported a total of {(format(md_beds, ',d'))} beds today out of which {md_covid_occupied_beds} — or {md_percent_covid_beds} percent — were occupied by COVID-19 patients, which is {beds_above_below} the national percentage. {md_percent_hospitals_critical_shortages} percent hospitals in the state are reporting a critical shortage, which is {shortage_below_above} the national percentage.\n
📊 📊 📊 
Percentage of hospitals with critical shortages nationally, today: {today_critical_shortage_hospitals} \n
Percentage of hospitals with critical shortages nationally, a week ago: {week_ago_critical_shortage_hospitals} \n
Percentage of hospitals with critical shortages in Maryland, today: {md_today_critical_shortage_hospitals} \n
Percentage of hospitals with critical shortages in Maryland, a week ago: {md_week_ago_critical_shortage_hospitals} \n
--- \n
🔗 View raw U.S. data: https://raw.githubusercontent.com/aadittambe/slack-bot/main/us_data.csv \n
🔗 View raw Maryland data: https://raw.githubusercontent.com/aadittambe/slack-bot/main/md_data.csv \n
ℹ️ Source: https://healthdata.gov/dataset/COVID-19-Reported-Patient-Impact-and-Hospital-Capa/6xf2-c3ie \n
📁 Analysis: https://github.com/aadittambe/slack-bot
"""

In [141]:
msg

'📢 Daily COVID-19 hospitalization update 📢\nOut of 758,088 available hospital beds in the country, 15,029 beds are occupied by COVID-19 patients today, which account for 1.98 percent of all beds.\n4.89 percent hospitals in the country are reporting critical staffing shortages today.\n\nMaryland reported a total of 11,639 beds today out of which 289 — or 2.48 percent — were occupied by COVID-19 patients, which is above the national percentage. 1.67 percent hospitals in the state are reporting a critical shortage, which is below the national percentage.\n\n📊 📊 📊 \nPercentage of hospitals with critical shortages nationally, today: 4.89 \n\nPercentage of hospitals with critical shortages nationally, a week ago: Not enough data to report \n\nPercentage of hospitals with critical shortages in Maryland, today: 1.67 \n\nPercentage of hospitals with critical shortages in Maryland, a week ago: Not enough data to report \n\n--- \n\n🔗 View raw U.S. data: https://raw.githubusercontent.com/aadittamb

## Send Slack message

In [144]:
slack_token = os.environ["SLACK_API_TOKEN"]
client = WebClient(token=slack_token)
try:
    response = client.chat_postMessage(
        channel="slack-bots",
        text=msg,
        unfurl_links=True, 
        unfurl_media=True
    )
    print("success!")
except SlackApiError as e:
    assert e.response["ok"] is False
    assert e.response["error"]
    print(f"Got an error: {e.response['error']}")

success!
